In [22]:
import unittest
import Model.BandaiDataset 
from Model.BandaiDataset import BandaiDataset
from Model.BandaiDataset import Motion


In [23]:
TEST_VEDIO_DIR = 'test/testfolder/'
TEST_FILE_NAME = 'dataset-1_walk_not-confident_001'
TEST_SET_FRAME = 10


In [1]:
class TestMotion(unittest.TestCase):
    def setup(self):
        self.motion = Motion()
    
    def test_input_motion(self):
        self.setUp()
        self.motion.input_motion(TEST_VEDIO_DIR, TEST_FILE_NAME)

    def test_input_label(self):
        self.motion.input_label(TEST_FILE_NAME)
        pass

    def test_adjust(self):
        self.motion.adjust(TEST_SET_FRAME)
        pass

    def test_get_motion_tensor(self):
        self.motion.get_motion_tensor()
        pass

    def test_getpose(self):
        pass

NameError: name 'unittest' is not defined

In [33]:
class TestBandaiDataset(unittest.TestCase):
    def setUp(self):
        self.dataset = BandaiDataset()
        
    def test_get_filenames(self):
        self.setUp()
        self.dataset.VIDEO_DIR = 'test/testfolder/'
        outfile = 'test/filenames.txt'
        self.dataset.get_filenames(outfile)

        self.assertEqual(self.dataset.filelist, ["dataset-1_walk_not-confident_001","dataset-1_walk_sad_002"])




In [34]:
test = TestBandaiDataset()
test.test_get_filenames()